In [38]:
%reset

import pandas as pd
from lxml import etree #for XML manipulation
import py2neo as neo
import re

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [39]:
#path to xml file as its saved outside my git repo:
path = '../../../DataFiles/dblp.xml'
#set up the parser with data type definition validation
#parser = etree.XMLParser(dtd_validation=True)

In [40]:
#this function returns (yields) a generator, so when the function is called,
#it is not run. This is so that we don't have to store the data in memory, we
#can simply work with it and discard it
def iterate_xml(path):
    doc = etree.iterparse(path,events=('start','end'),dtd_validation=True)
    _, root = next(doc)
    start_tag = None
    for event, element in doc:
        if event == 'start' and start_tag is None:
            start_tag = element.tag
        if event == 'end' and element.tag == start_tag:
            yield element
            start_tag = None
            root.clear()

In [41]:
generator = iterate_xml(path)
generator

<generator object iterate_xml at 0x7efc2241a258>

In [67]:
i = 0
for event in generator:
    print('_______________________')
    print('Key: ' + str(event.attrib['key']))
    xml = etree.tostring(event.find('title')).decode('utf-8') 
    title = re.search('<title>(.+?)</title>',xml).group(1)
    print(title)
    for author in auth_list:
        xml = etree.tostring(author).decode('utf-8')
        author = re.search('<author>(.+?)</author>',xml).group(1)
        print(author)

    i = i+1
    if i>30:
        break

_______________________
Key: journals/acta/Martin83
A General Proof Rule for Procedures in Predicate Transformer Semantics.
Bruce Russell
_______________________
Key: journals/acta/AdamsGV79
An Experimentally Validated Model of the Paging Drum.
Bruce Russell
_______________________
Key: journals/acta/BookC76
Inherently Nonplanar Automata.
Bruce Russell
_______________________
Key: journals/acta/CoffmanDW02
Packing rectangles in a strip.
Bruce Russell
_______________________
Key: journals/acta/AroraJSMC06
Throughput analysis in wireless networks with multiple users and multiple channels.
Bruce Russell
_______________________
Key: journals/acta/Deussen78
A Unified Approach to the Generation and the Acception of Formal Languages.
Bruce Russell
_______________________
Key: journals/acta/GergelyU82
A Theory of Interactive Programming.
Bruce Russell
_______________________
Key: journals/acta/LiZPDZ16
Analysis of dual-hop and multiple relays cooperative truncated ARQ with relay selection in W

Connect to neo4j server

In [80]:
#note, server must be running on localhost 
#sudo service neo4j start
from py2neo import Graph, Node

graph = Graph('bolt://localhost:7687', auth=('neo4j','ItIsABadIdeaToHardCodePasswords'))

In [ ]:
#Set Uniqueness Constraints on Nodes
graph.schema.create_uniqueness_constraint('Article', 'key')
graph.schema.create_uniqueness_constraint('Author', 'name')

Reset the generator

In [81]:
generator = iterate_xml(path)

#delete all nodes, used when testing out the program to start fresh each time
#graph.delete_all()


i = 0
for event in generator:
    print('_______________________')
    key = str(event.attrib['key'])
    print('Key: ' + key)
    xml = etree.tostring(event.find('title')).decode('utf-8') 
    title = re.search('<title>(.+?)</title>',xml).group(1)
    print(title)
    #save the node
    #article = Node('Article', key=key, title=title)
    article = graph.merge('Article',property_key='key',property_value=key)
    article['title']=title
    article.push()
    
    auth_list = event.findall('author')    
    for author in auth_list:
        xml = etree.tostring(author).decode('utf-8')
        name = re.search('<author>(.+?)</author>',xml).group(1)
        print(name)
        #save the node
        #author_node = Node('Author', name=name, department=None)
        author = graph.merge('Author',property_key='name',property_value=name)
        author.push()
        
    i = i+1
    if i>30:
        break

_______________________
Key: journals/acta/Saxena96
Parallel Integer Sorting and Simulation Amongst CRCW Models.


TypeError: merge() got an unexpected keyword argument 'property_key'